<a href="https://colab.research.google.com/github/ahmadmasood43/Glioblastoma-Detection-and-VR-Conversion/blob/main/Segmentation_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Path to the folders containing the tumor and non-tumor images
tumor_folder = '/content/gdrive/MyDrive/new_fyp_dataset_folder/yes'

# Path to the CSV files containing the annotations
tumor_annotations_csv = '/content/gdrive/MyDrive/new_fyp_dataset_folder/annotation_fyp_all_yes.csv'

In [3]:
# Define the input size for the model
input_width = 224
input_height = 224

In [4]:
# Load and preprocess the images and annotations
def load_data(image_folder, annotations_csv):
    # Load the annotations from the CSV file
    annotations = pd.read_csv(annotations_csv)
    
    image_filenames = os.listdir(image_folder)
    images = []
    masks = []
    
    for filename in image_filenames:
        # Load the image
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)
        print(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (input_width, input_height))
        images.append(image)
        
        # Get the corresponding annotation filename
        annotation_filename = filename.replace('.png', '.csv')
        annotation_path = os.path.join(annotations_csv, annotation_filename)
        
        # Load the corresponding annotation mask
        annotation_mask = np.zeros((input_height, input_width, 1), dtype=np.uint8)
        if os.path.isfile(annotation_path):
            annotation_df = pd.read_csv(annotation_path)
            for _, row in annotation_df.iterrows():
                x = int(row['x'])
                y = int(row['y'])
                w = int(row['w'])
                h = int(row['h'])
                annotation_mask[y:y+h, x:x+w] = 255
        
        masks.append(annotation_mask / 255.0)  # Normalize to [0, 1]
    
    images = np.array(images)
    masks = np.array(masks)
    return images, masks

# Load the tumor and non-tumor images and masks
tumor_images, tumor_masks = load_data(tumor_folder, tumor_annotations_csv)
# Concatenate the tumor and non-tumor data
images = np.concatenate([tumor_images], axis=0)
masks = np.concatenate([tumor_masks], axis=0)

# Split the data into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)


/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/77 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/192 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/4 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/164 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/119 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/1 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/89 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/185 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/228 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/154 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/217 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/90 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/92 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/199 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/220 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/200 yes.jpg


In [5]:
# Load the tumor and non-tumor images and masks
tumor_images, tumor_masks = load_data(tumor_folder, tumor_annotations_csv)

# Concatenate the tumor and non-tumor data
images = np.concatenate([tumor_images], axis=0)
masks = np.concatenate([tumor_masks], axis=0)


/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/77 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/192 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/4 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/164 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/119 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/1 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/89 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/185 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/228 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/154 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/217 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/90 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/92 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/199 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/220 yes.jpg
/content/gdrive/MyDrive/new_fyp_dataset_folder/yes/200 yes.jpg


In [6]:
# Split the data into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

In [7]:
# Define the U-Net model architecture
inputs = Input(shape=(input_height, input_width, 3))
conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)

In [8]:

from tensorflow.keras.layers import Dropout

conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
pool1 = Dropout(0.5)(pool1)

conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
pool2 = Dropout(0.5)(pool2)

conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
pool3 = Dropout(0.5)(pool3)

conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
pool4 = Dropout(0.5)(pool4)

conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)

up6 = UpSampling2D(size=(2, 2))(conv5)
up6 = Conv2D(256, 2, activation='relu', padding='same')(up6)
merge6 = tf.keras.layers.Concatenate(axis=3)([conv4, up6])
conv6 = Conv2D(256, 3, activation='relu', padding='same')(merge6)
conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)

up7 = UpSampling2D(size=(2, 2))(conv6)
up7 = Conv2D(128, 2, activation='relu', padding='same')(up7)
merge7 = tf.keras.layers.Concatenate(axis=3)([conv3, up7])
conv7 = Conv2D(128, 3, activation='relu', padding='same')(merge7)
conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)

up8 = UpSampling2D(size=(2, 2))(conv7)
up8 = Conv2D(64, 2, activation='relu', padding='same')(up8)
merge8 = tf.keras.layers.Concatenate(axis=3)([conv2, up8])
conv8 = Conv2D(64, 3, activation='relu', padding='same')(merge8)
conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)

up9 = UpSampling2D(size=(2, 2))(conv8)
up9 = Conv2D(32, 2, activation='relu', padding='same')(up9)
merge9 = tf.keras.layers.Concatenate(axis=3)([conv1, up9])
conv9 = Conv2D(32, 3, activation='relu', padding='same')(merge9)
conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

In [ ]:
conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
pool1 = Dropout(0.5)(pool1)

conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
pool2 = Dropout(0.5)(pool2)

conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
pool3 = Dropout(0.5)(pool3)

conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
pool4 = Dropout(0.5)(pool4)

conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)

up6 = UpSampling2D(size=(2, 2))(conv5)
up6 = Conv2D(256, 2, activation='relu', padding='same')(up6)
merge6 = tf.keras.layers.Concatenate(axis=3)([conv4, up6])
conv6 = Conv2D(256, 3, activation='relu', padding='same')(merge6)
conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)

up7 = UpSampling2D(size=(2, 2))(conv6)
up7 = Conv2D(128, 2, activation='relu', padding='same')(up7)
merge7 = tf.keras.layers.Concatenate(axis=3)([conv3, up7])
conv7 = Conv2D(128, 3, activation='relu', padding='same')(merge7)
conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)

up8 = UpSampling2D(size=(2, 2))(conv7)
up8 = Conv2D(64, 2, activation='relu', padding='same')(up8)
merge8 = tf.keras.layers.Concatenate(axis=3)([conv2, up8])
conv8 = Conv2D(64, 3, activation='relu', padding='same')(merge8)
conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)

up9 = UpSampling2D(size=(2, 2))(conv8)
up9 = Conv2D(32, 2, activation='relu', padding='same')(up9)
merge9 = tf.keras.layers.Concatenate(axis=3)([conv1, up9])
conv9 = Conv2D(32, 3, activation='relu', padding='same')(merge9)
conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)


In [ ]:
# Compile the model
model = Model(inputs=inputs, outputs=conv10)
model.compile(optimizer=Adam(learning_rate=1e-6), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train the model with early stopping
history = model.fit(train_images, train_masks, validation_data=(val_images, val_masks), batch_size=32, epochs=5, callbacks=[early_stopping])

Epoch 1/5
7/7 [==============================] - 456s 64s/step - loss: 2.9936 - accuracy: 0.0632 - val_loss: 3.2276 - val_accuracy: 0.0720
Epoch 2/5
7/7 [==============================] - 447s 64s/step - loss: 2.8953 - accuracy: 0.0742 - val_loss: 3.1434 - val_accuracy: 0.0829
Epoch 3/5
7/7 [==============================] - 448s 64s/step - loss: 2.7963 - accuracy: 0.0878 - val_loss: 3.0589 - val_accuracy: 0.0961
Epoch 4/5
4/7 [================>.............] - ETA: 3:13 - loss: 2.6203 - accuracy: 0.1037

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

# Define the input size for the model
input_width = 224
input_height = 224

# Path to the folder containing the test images
test_folder = '/content/gdrive/MyDrive/new_fyp_dataset_folder/testing'

# Get the list of image filenames in the test folder
image_filenames = os.listdir(test_folder)

# Perform inference on test images
for filename in image_filenames:
    # Load the test image
    image_path = os.path.join(test_folder, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_width, input_height))
    test_image = np.expand_dims(image, axis=0)

    # Perform prediction on the test image
    prediction = model.predict(test_image)

    # Convert the predicted mask to binary
    mask = (prediction[0, :, :, 0] > 0.5).astype(np.uint8)

    # Resize the mask to match the shape of the colored_image
    resized_mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    # Apply the boolean indexing to each channel of the image array separately
    colored_image = image.copy()
    colored_image[resized_mask == 1] = [255, 0, 0]  # Change the color to red

    # Display the original image and the colored image side by side
    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(image)
    axs[0].set_title('Original Image')
    axs[1].imshow(colored_image)
    axs[1].set_title('Image with Tumor Regions')
    plt.show()